In [28]:
from __future__ import print_function
import torch
import numpy as np

In [105]:
#Lesson One - Tensors and Numpy Integrations
'''An uninitialized matrix is declared,
but does not contain definite known values before it is used.
When an uninitialized matrix is created, whatever values were 
in the allocated memory at the time will appear as the initial values.'''

#Uninitialized
matrix = torch.empty(5,3)

#Random
matrix = torch.rand(5,3)
# print(matrix)

#np.zeros but contains a long

x = torch.zeros(5,3, dtype=torch.long)
# print(x)

#create a tensor with manual data - set type equal to float
x = torch.tensor([5.5,3], dtype = torch.float)
# print(x)

#create a tensor based on the previous tensor
x = x.new_ones(3,3,dtype=torch.double)
#fill the tensor with random doubles
x = torch.randn_like(x, dtype=torch.double)
#print size of array
# x.size()
#add two tensors of the same size
y = torch.rand(3,3, dtype=torch.double)
res = torch.empty(3,3)
torch.add(x,y, out=res)
#operation that mutates a tensor in place is post fixed with an _
# res.copy_(y)

#indexing
# print(x)
x[:,-1] #last column
x = torch.rand(4,4)
#resizing a tensor
# (x.view(-1,8))

#one element tensor
# x = torch.randn(1)
#prints the one item of the one element tensor
# (x.item())

#converting to a np array

a = torch.zeros(3,3, dtype = torch.double)
a.add_(2)
b = a.numpy()

#converting np array to torch tensor
import numpy as np

a = np.zeros((3,3))
b = torch.from_numpy(a)
print(b.to(torch.int))


tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)
False


In [184]:
# Lesson 2 - Linear Regression
#predicting yields of apples and oranges depending on input variables/features

#in linear regression - each target variable is estimated to be a weighted sum of the input variables, offset by some constant known as bias
#usually it is just some weights multiplied by input variables, and a constant is added to it in case there is a
#discrepancy in the training data

import numpy as np
import torch

#Inputs (temperature, rainfall, humidity)

inp = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                   [102,43,37],
                  [69,96,70]], dtype='double')
trg = np.array([[56,70],
                  [81,101],
                  [119,133],
                   [22,37],
                  [103,119]], dtype='double')
#you can get a column by doing [:,0]
#Targets (apples, oranges) inital data before the prediction
# targets = np.array([[56,70],[81,101],[119,133],[22,37], [103,119]], dtype='double')
inputs = torch.from_numpy(inp)
targets = torch.from_numpy(trg)

#we can create some random weights - but since there are three input types, (temp, rainfall, hum) we need three weights


#weights are matrices, they are random for now
weights = torch.randn((2,3), requires_grad=True, dtype=torch.double)

#biases are vectors, they are random as well. They will be the constants we add to the weights and the inps
biases = torch.randn((2), requires_grad=True, dtype=torch.double)


In [185]:

# print(x * y)

# # @ gives the final value of a matrix multiplication set
# print(x @ y)
# print(weights.t())
# print(inp)


#structure of weights matrix = (inputs values, target values)

def model(x):
    return x @ weights.t() + biases 
predictions = model(inputs)

def MSE(t1, t2):
    #evaluate how well model is performing
    diff = t1 - t2
    diff_sqr = diff ** 2
    #get the average of the squared matrix - sum of all values / len of matrix
    #numel gets number of elements , or diff.size()[0] * diff.size()[1]
    return torch.sum(diff_sqr) / torch.numel(diff_sqr)
loss = MSE(predictions, targets)
print("inital loss", loss)
#interpretation: On average, each of model's predictions is off by the sqrt of the loss(for example 255)
#tells how much info it is losing - when random - very much loss


#Loss is a quadratic function of the weights and biases
#if you increase / decrease the weights too much then the loss will be too high 


#Positive Weights and Positive Gradient
#make sure to decrease slightly, otherwise the loss will be bigger
#increasing the elements value will increase the loss
#However, if you decrease the element's value, then the loss will decrease

#Positive Weights and Negative Gradient
#make sure to decrease slightly, otherwise the loss will be bigger
#increasing the element will decrease the loss
#decreasing the elemnt will increase the loss



inital loss tensor(5089.0522, dtype=torch.float64, grad_fn=<DivBackward0>)


In [186]:
preds = model(inputs)
print(preds)

tensor([[ 56.9343, 149.4976],
        [ 77.5267, 195.1484],
        [ 66.6947, 254.0304],
        [ 69.2831, 134.9022],
        [ 70.2167, 192.5821]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [187]:
loss = MSE(preds,targets)
loss

tensor(5089.0522, dtype=torch.float64, grad_fn=<DivBackward0>)

In [188]:
# print(weights, biases)
# #make sure that pytorch doesn't track or modify the gradients
# with torch.no_grad():
#     #subtracting because the value is a negative, so it is basically adding the abs. A clearer way of doing it would
#     #be abs(weights) * 1e-5 (which is a very small number)
#     weights-=weights.grad * 1e-5
#     biases -= biases.grad * 1e-5

#     weights.grad.zero_()
#     biases.grad.zero_()

In [189]:
#as you can see, the loss has gone down by a little more than a 1000. This is becasue we are decreasing the value
#of the weights decreases the value of the  
preds = model(inputs)
loss = MSE(preds, targets)
print(loss)

tensor(5089.0522, dtype=torch.float64, grad_fn=<DivBackward0>)


In [190]:
#train for 100 epochs (iterations)
for i in range(100):
    preds = model(inputs)
    loss = MSE(preds, targets)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero_()
        biases.grad.zero_()

In [191]:
preds = model(inputs)
loss = MSE(preds, targets)
print(loss)

tensor(205.1980, dtype=torch.float64, grad_fn=<DivBackward0>)


In [192]:
preds

tensor([[ 62.4288,  72.5737],
        [ 87.2248,  95.6206],
        [ 98.8187, 140.7981],
        [ 51.8071,  50.7327],
        [ 92.7226, 102.1012]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [193]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]], dtype=torch.float64)